<a href="https://colab.research.google.com/github/hubertwel/paragraph-similarity/blob/main/paragraph-similarity/paragraph_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# UNDER CONSTRUCTION
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_multiple_whitespaces
from gensim.parsing.preprocessing import remove_stopwords, strip_short
from gensim.parsing.preprocessing import strip_non_alphanum, split_alphanum
from gensim.parsing.preprocessing import strip_punctuation
from gensim.utils import simple_preprocess
from gensim.utils import RULE_KEEP, RULE_DISCARD, RULE_DEFAULT
from gensim.test.utils import get_tmpfile
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
import numpy as np
import os
import smart_open
import csv
import collections
import random
import re
import optuna

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        csv_reader = csv.DictReader(f, quoting=csv.QUOTE_ALL)
        CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_multiple_whitespaces]
        for i, row in enumerate(csv_reader):
          line = row['x']
          line = remove_urls(line)
          line_list = preprocess_string(line, CUSTOM_FILTERS)
          line = " ".join(line_list)
          tokens = simple_preprocess(line)
          if tokens_only:
            yield tokens
          else:
            # For training data, add tags
            yield TaggedDocument(tokens, [i])

def remove_urls(text):
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
  return text

def trim_rule(word, count, min_count):
    stop_words = set(stopwords.words('english')) 
    # This rule is only used to prune vocabulary during the current method call,
    # so that documents can be printed with stopwords and with words of any length
    if ((word in stop_words) or (len(word) < 3)):
        return utils.RULE_DISCARD  # throw out
    else:
        return utils.RULE_DEFAULT  # apply default rule, i.e. min_count

# Define an objective function to be maximized
def objective(trial):
  classifier_name = trial.suggest_categorical("classifier", ["LogReg"])
  # Setup values for the hyperparameters:
  if classifier_name == 'LogReg':
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    clf = linear_model.LogisticRegression(C=logreg_c)
  # Scoring method:
    k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
    score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring='accuracy')
    accuracy = score.mean()
    return accuracy

def main():    
  global X_train, y_train

  # Set file names for train and test data
  # test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
  test_data_dir = os.path.join('/content/drive/MyDrive/Colab Notebooks/data/', 'gouvfr', 'CorpusRandomTwitter')
  print("test_data_dir: %s" % test_data_dir)
  train_file = os.path.join(test_data_dir, 'randomtweets3.txt')
  test_file = os.path.join(test_data_dir, 'randomtweets4.txt')

  train_corpus = list(read_corpus(train_file, tokens_only=True))
  train_corpus_tagged = list(read_corpus(train_file))
  test_corpus = list(read_corpus(test_file, tokens_only=True))

  print('train_corpus the beginning: ', train_corpus[:2])
  print('train_corpus length %d' % len(train_corpus))
  print('train_corpus_tagged the beginning: ', train_corpus_tagged[:2])
  print('test_corpus the beginning: ', test_corpus[:2])
  print('test_corpus length %d' %len(test_corpus))
  print()

  # Build a vocabulary
  model = Doc2Vec(dm=0, vector_size=80, min_count=3, epochs=50, hs=1, dbow_words=1, trim_rule=trim_rule)
  model.build_vocab(train_corpus_tagged)

  # Train the model on the corpus
  model.train(train_corpus_tagged, total_examples=model.corpus_count, epochs=model.epochs)
  print()

  # Assessing the model
  ranks = []
  first_ranks = []
  second_ranks = []
  inferred_vectors = []
  for doc_id in range(len(train_corpus_tagged)):
    inferred_vector = model.infer_vector(train_corpus_tagged[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    # sanity check (self-similarity)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    first_ranks.append(sims[0][0])
    inferred_vectors.append(inferred_vector)
        
  print('sims: ', sims[:3])
  print('ranks: ', ranks)
  counter = collections.Counter(ranks)
  print(counter)
  print()

  # Testing the model with one random document
  # Pick a random document from the test corpus and infer a vector from the model
  doc_id = random.randint(0, len(test_corpus) - 1)
  inferred_vector = model.infer_vector(test_corpus[doc_id])
  sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
  print('RANDOM TEST DOCUMENT ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))

  # Compare and print the most/second-most/third-most/median/least similar documents from the train corpus
  print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
  for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('THIRD-MOST', 2), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus_tagged[sims[index][0]].words)))

  # Save the model
  filename = get_tmpfile("parsim_doc2vec_model")
  model.save(filename)

  # Assessing the model
  ranks_test = []
  first_ranks_test = []
  inferred_vectors_test = []
  for doc_id in range(len(test_corpus)):
    inferred_vector_test = model.infer_vector(test_corpus[doc_id])
    sims_test = model.dv.most_similar([inferred_vector_test], topn=len(model.dv))
    # sanity check (self-similarity)
    rank_test = [docid for docid, sim in sims_test].index(doc_id)
    ranks_test.append(rank_test)
    first_ranks_test.append(sims_test[0][0])
    inferred_vectors_test.append(inferred_vector_test)

  print('first_ranks_test: ', first_ranks_test)

  # Prepare vectors for cross validatiom
  tags_array_train = np.array(first_ranks)
  vectors_2Darray_train = np.array(inferred_vectors)
  tags_array_test = np.array(first_ranks_test)
  vectors_2Darray_test = np.array(inferred_vectors_test)
  y_train, X_train = tags_array_train, vectors_2Darray_train
  y_test, X_test = tags_array_test, vectors_2Darray_test

  # Create Optuna study
  study = optuna.create_study(direction="maximize")
  study.optimize(objective, n_trials=10)
  
  # Cross validation
  clf = LogisticRegression(solver='liblinear', max_iter=300, class_weight='balanced', multi_class='auto')
  k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
  score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring='accuracy')
  print('score: ', score)
  print('Validation accuracy: {}'.format(round(np.mean(score)*100, 3)))
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print("Test accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

main()

test_data_dir: /content/drive/MyDrive/Colab Notebooks/data/gouvfr/CorpusRandomTwitter
train_corpus the beginning:  [['rt', 'americanidol', 'watch', 'as', 'ddlovato', 'gives', 'an', 'amazing', 'performance', 'of', 'her', 'single', 'stonecold', 'demionidol'], ['rt', 'dipti_varun', 'varun', 'dhawan', 'junaid', 'is', 'rookie', 'cop', 'and', 'heâ', 'kind', 'of', 'discovering', 'himself', 'as', 'the', 'movie', 'goes', 'on', 'varun_dvn', 'dishoom']]
train_corpus length 1000
train_corpus_tagged the beginning:  [TaggedDocument(words=['rt', 'americanidol', 'watch', 'as', 'ddlovato', 'gives', 'an', 'amazing', 'performance', 'of', 'her', 'single', 'stonecold', 'demionidol'], tags=[0]), TaggedDocument(words=['rt', 'dipti_varun', 'varun', 'dhawan', 'junaid', 'is', 'rookie', 'cop', 'and', 'heâ', 'kind', 'of', 'discovering', 'himself', 'as', 'the', 'movie', 'goes', 'on', 'varun_dvn', 'dishoom'], tags=[1])]
test_corpus the beginning:  [['live', 'morning', 'news', 'weather', 'and', 'traffic', 'updates',

[I 2021-02-15 01:35:17,245] A new study created in memory with name: no-name-d801b486-775b-4a6d-b6fb-64fe7ef3e7d3


first_ranks_test:  [391, 763, 943, 492, 994, 955, 349, 714, 238, 131, 676, 886, 814, 656, 662, 493, 582, 325, 87, 69, 142, 893, 208, 412, 340, 82, 642, 580, 507, 235, 786, 800, 647, 42, 494, 7, 145, 496, 262, 733, 375, 893, 697, 914, 485, 355, 671, 252, 76, 82, 206, 250, 535, 665, 361, 68, 413, 546, 922, 789, 621, 567, 174, 939, 64, 545, 932, 240, 738, 251, 925, 764, 45, 270, 516, 692, 285, 76, 131, 546, 159, 404, 399, 336, 503, 82, 959, 955, 647, 709, 87, 733, 221, 258, 768, 924, 131, 481, 147, 778, 634, 194, 131, 17, 660, 103, 141, 906, 657, 514, 791, 206, 95, 163, 506, 921, 117, 994, 78, 769, 105, 886, 700, 197, 314, 340, 131, 131, 463, 764, 225, 788, 165, 271, 576, 352, 557, 683, 438, 660, 292, 451, 226, 500, 715, 633, 177, 334, 201, 225, 607, 131, 42, 977, 659, 776, 277, 453, 87, 287, 566, 682, 399, 182, 352, 780, 701, 297, 414, 397, 621, 261, 330, 364, 720, 278, 223, 360, 775, 967, 874, 507, 237, 591, 199, 36, 909, 682, 426, 571, 560, 335, 633, 208, 285, 170, 234, 604, 430, 369, 

[I 2021-02-15 01:35:58,952] Trial 0 finished with value: 0.016 and parameters: {'classifier': 'LogReg', 'logreg_c': 2.7839893556013824e-06}. Best is trial 0 with value: 0.016.
[I 2021-02-15 01:36:48,030] Trial 1 finished with value: 0.12300000000000003 and parameters: {'classifier': 'LogReg', 'logreg_c': 423300.6158379168}. Best is trial 1 with value: 0.12300000000000003.
[I 2021-02-15 01:37:35,163] Trial 2 finished with value: 0.12600000000000003 and parameters: {'classifier': 'LogReg', 'logreg_c': 14864.304246725385}. Best is trial 2 with value: 0.12600000000000003.
[I 2021-02-15 01:37:52,503] Trial 3 finished with value: 0.10600000000000001 and parameters: {'classifier': 'LogReg', 'logreg_c': 0.23552826173360714}. Best is trial 2 with value: 0.12600000000000003.
[I 2021-02-15 01:38:39,564] Trial 4 finished with value: 0.12700000000000003 and parameters: {'classifier': 'LogReg', 'logreg_c': 23.18007155068059}. Best is trial 4 with value: 0.12700000000000003.
[I 2021-02-15 01:39:26,93

score:  [0.15 0.18 0.13 0.13 0.13 0.08 0.1  0.18 0.1  0.09]
Validation accuracy: 12.7
Test accuracy: 0.550


In [17]:
pip install -U scikit-learn

     |████████████████████████████████| 22.2MB 1.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [1]:
pip install optuna

     |████████████████████████████████| 296kB 11.1MB/s 
     |████████████████████████████████| 1.1MB 22.0MB/s 
     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 481kB 33.3MB/s 
     |████████████████████████████████| 143kB 38.4MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 
     |████████████████████████████████| 51kB 3.4MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=309316069b483ad916f9ec051c88ab2a31c6423d2682b9e1cb724c692bc6f9f5
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5a1b1e2363ed68392d292d215facf588d5448198edd8078bc1
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=e14d0f5f0564f190f342d775114ec7b7a73b093e83d86dcfecf2ba478cf930cd
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sh

In [ ]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
diabetes = datasets.load_diabetes()
X = diabetes.data[:150]
y = diabetes.target[:150]
print('X: ', X)
print('y: ', y)
y = diabetes.target[:150]
lasso = linear_model.Lasso()
print(cross_val_score(lasso, X, y, cv=3))


X:  [[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990842
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06832974
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286377
  -0.02593034]
 ...
 [-0.05637009 -0.04464164  0.09295276 ...  0.02545259  0.02605609
   0.04034337]
 [-0.06000263  0.05068012  0.01535029 ... -0.00259226 -0.03075121
  -0.0010777 ]
 [-0.04910502  0.05068012 -0.00512814 ...  0.07120998  0.06123791
  -0.03835666]]
y:  [151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 279.
  9

In [3]:
pip install --pre --upgrade gensim

     |████████████████████████████████| 24.0MB 1.8MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
